In [1]:
# #In lambdalabs jupyter lab instance, run these:
# pip install transformers
# pip install tf-keras
# pip install --upgrade "numpy<2"
# pip install datasets
# pip install --upgrade datasets pillow
# pip install --upgrade "accelerate>=0.26.0"
# #then check dependency warnings
# pip check
# #if any issues run these SEPARATELY!
# pip install debugpy
# c
# # then install these
# sudo apt-get update
# sudo apt-get install python3-cairo

In [1]:
import os
import numpy as np
import subprocess
import tensorflow as tf
import torch
import torchvision.transforms as T
from datasets import load_dataset, Image as DatasetsImage
from functools import partial
from io import BytesIO
from transformers import (
    AutoImageProcessor, 
    AutoModelForImageClassification, 
    EarlyStoppingCallback,
    TrainingArguments, 
    Trainer
)
from torch import nn
from PIL import Image, ImageOps, ExifTags, UnidentifiedImageError

2025-03-23 13:58:37.628409: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742738317.651580    4431 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742738317.659552    4431 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742738317.678797    4431 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742738317.678880    4431 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742738317.678889    4431 computation_placer.cc:177] computation placer alr

In [ ]:
# # --------------------------
# # 0. GPU Environment Setup for Multi-GPU Optimization (GPUs 0)
# # --------------------------
# # Limit process to specific GPUs
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# print("Process restricted to GPUs:", os.environ["CUDA_VISIBLE_DEVICES"])

# # Ensure pip executables are available
# os.environ["PATH"] = f"{os.path.expanduser('~/.local/bin')}:" + os.environ["PATH"]

# # Enable memory growth for TensorFlow
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         print("Memory growth enabled on GPUs.")
#     except RuntimeError as e:
#         print("Error configuring GPUs:", e)
# print("GPUs available to this process (as seen by TensorFlow):", tf.config.list_physical_devices('GPU'))

# # Optional: Monitor current GPU usage
# gpu_usage = subprocess.check_output(["nvidia-smi"]).decode("utf-8")
# print("Current GPU usage:\n", gpu_usage)

In [2]:
# --------------------------
# 1. Load Model (V4 base)
# --------------------------
model_path = "/home/ubuntu/MLexpressionsStorage/vit_final_independent_V4"
model = AutoModelForImageClassification.from_pretrained(model_path)
processor = AutoImageProcessor.from_pretrained(model_path)
model.eval()

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [3]:
# --------------------------
# 2. Prepare Dataset 
# --------------------------
dataset = load_dataset(
    "imagefolder",
    data_dir="/home/ubuntu/MLexpressionsStorage/img_datasets/combo_ferckja_dataset",
    split="train"
)

# ----- Label Reconciliation for Locally Pre-Trained Model -----
# Assume 'dataset' is already loaded from your image folder dataset.
# (For example, using: dataset = load_dataset("imagefolder", data_dir="...", split="train"))

# Optionally, inspect the original unique labels:
try:
    original_labels = dataset.unique("label")
    print("Original unique labels:", original_labels)
except Exception as e:
    print("Unable to retrieve original unique labels:", e)

# Define a mapping from eight original labels to seven target classes
# Update mapping using lowercase keys
label_mapping = {
    'anger': 'Angry', 'contempt': 'Disgust', 'disgust': 'Disgust',
    'fear': 'Fear', 'happiness': 'Happy', 'sadness': 'Sad',
    'surprise': 'Surprise', 'neutral': 'Neutral'
}

# Numerical mapping for the pre-trained model's labels.
num_mapping = {
    'Angry': 0, 'Disgust': 1, 'Fear': 2, 'Happy': 3,
    'Sad': 4, 'Surprise': 5, 'Neutral': 6
}

def reconcile_labels(example):
    # Convert the label to a lowercase string.
    if isinstance(example["label"], int):
        # Convert integer label to string using the dataset's features.
        original = dataset.features["label"].int2str(example["label"]).strip().lower()
    else:
        original = example["label"].strip().lower()
    
    # Map the original label to the new label.
    mapped = label_mapping.get(original)
    if mapped is None:
        # Mark unrecognized labels as -1 for filtering.
        example["label"] = -1
    else:
        # Convert to numerical label.
        example["label"] = num_mapping[mapped]
    return example

# Apply the mapping function.
dataset = dataset.map(reconcile_labels)
# Filter out any examples with unrecognized labels.
dataset = dataset.filter(lambda x: x["label"] != -1)

# Verify the final set of labels (should be only 7 unique values).
mapped_unique_labels = dataset.unique("label")
print("Mapped unique labels (numerical):", mapped_unique_labels)

Original unique labels: [0, 1, 2, 3, 4, 5, 6, 7]


Filter: 100%|██████████| 37081/37081 [01:19<00:00, 468.21 examples/s]

Mapped unique labels (numerical): [0, 1, 2, 3, 6, 4, 5]


In [4]:
# --------------------------
# 3. Stronger Augmentation Pipeline
# --------------------------
data_augment = T.Compose([
    T.RandomResizedCrop(224, scale=(0.8, 1.0)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomVerticalFlip(p=0.2),
    T.RandomRotation(15),
    T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.05),
    T.RandomGrayscale(p=0.1)
])

def transform(example):
    image = example["image"]
    # Ensure image is loaded as a PIL Image
    if not isinstance(image, Image.Image):
        image = Image.open(image)
    # Convert to RGB if needed
    if image.mode != "RGB":
        image = image.convert("RGB")
    # Apply augmentation
    image = data_augment(image)
    # Process image using the processor (this produces a tensor in the correct format)
    inputs = processor(image, return_tensors="pt")
    # Remove the extra batch dimension (we process images individually)
    inputs = {k: v.squeeze(0) for k, v in inputs.items()}
    # Add the label back
    inputs["labels"] = example["label"]
    return inputs

# Apply the transformation to the dataset while preserving the 'label' column.
dataset = dataset.map(
    partial(transform),
    remove_columns=[col for col in dataset.column_names if col not in ["label", "image"]]
)

# --------------------------
# New Step: Compute Unique Labels from the Final Preprocessed Column
# --------------------------
unique_labels = dataset.unique("labels")
print("Unique labels in dataset:", unique_labels)

Map: 100%|██████████| 37081/37081 [07:52<00:00, 78.42 examples/s] 

Unique labels in dataset: [0, 1, 2, 3, 6, 4, 5]


In [5]:
# --------------------------
# 4. Train-Validation Split
# --------------------------
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

In [6]:
# --------------------------
# 5. Define Training Arguments for Robust Fine-Tuning
# --------------------------
training_args = TrainingArguments(
    output_dir="./vit_finetuned_v5",    # Directory to save checkpoints and the final model
    eval_strategy="epoch",           # Evaluate at the end of each epoch
    save_strategy="epoch",                 # Save checkpoint at each epoch
    learning_rate=3e-5,                    # A conservative learning rate for fine-tuning
    per_device_train_batch_size=8,         # Adjust based on your CPU memory limits
    per_device_eval_batch_size=8,
    num_train_epochs=5,                    # Fine-tune for a few epochs (adjust as needed)
    load_best_model_at_end=True,           # Automatically load the best model when training finishes
    metric_for_best_model="accuracy",      # Monitor accuracy for best model selection
    logging_dir="./logs",                  # Directory for TensorBoard logs
    warmup_ratio=0.2,                      # Increased for smoother ramp-up
    save_total_limit=2
)

In [7]:
# --------------------------
# 6. Define a Compute Metrics Function for Evaluation
# --------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": (predictions == labels).mean()}

In [8]:
# --------------------------
# 7. Initialize Trainer with EarlyStopping
# --------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.674800,0.693736,0.754213
2,0.438400,0.734173,0.771201
3,0.166300,1.020023,0.777134
4,0.044800,1.218053,0.785223


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [9]:
# --------------------------
# 8. Save Final Model
# --------------------------
torch.save(model.state_dict(), '/home/ubuntu/MLexpressionsStorage/final_model_V6.pth')
model.save_pretrained("/home/ubuntu/MLexpressionsStorage/vit_final_independent_V6")
processor.save_pretrained("/home/ubuntu/MLexpressionsStorage/vit_final_independent_V6")

['/home/ubuntu/MLexpressionsStorage/vit_final_independent_V6/preprocessor_config.json']